# torch.nn.utils.rnn
All contents are referenced to below link. `PaddedSequence` is a just `torch.Tensor` applied to padding technique.

* link : https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-11_7_PackedSequence.ipynb

![Alt text](https://github.com/deeplearningzerotoall/PyTorch/raw/3a4fc62cb1225730c8a90e32665cb8b5021329f1/figures/sequence.png)

### Setup

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_sequence, pack_padded_sequence
from pprint import pprint

### Example data

In [2]:
data = ['hello world', 'midnight', 'calculation', 'path', 'short circuit']

char_set = ['<pad>'] + sorted(set(''.join(data)))
char2idx = {token : idx for idx, token in enumerate(char_set)}
indices = list(map(lambda string: torch.tensor([char2idx.get(char) for char in string]), data)) # list of Tensors
pprint(indices)

[tensor([ 7,  5,  9,  9, 12,  1, 18, 12, 14,  9,  4]),
 tensor([10,  8,  4, 11,  8,  6,  7, 16]),
 tensor([ 3,  2,  9,  3, 17,  9,  2, 16,  8, 12, 11]),
 tensor([13,  2, 16,  7]),
 tensor([15,  7, 12, 14, 16,  1,  3,  8, 14,  3, 17,  8, 16])]


## pad_sequence
link : https://pytorch.org/docs/stable/nn.html#pad-sequence

In [3]:
padded_indices_wbf = pad_sequence(indices, batch_first=True)
padded_indices_wobf  = pad_sequence(indices)

print(padded_indices_wbf, padded_indices_wbf.size()) # (batch, seq_len)
print(padded_indices_wobf, padded_indices_wobf.size()) # (seq_len, batch)

tensor([[ 7,  5,  9,  9, 12,  1, 18, 12, 14,  9,  4,  0,  0],
        [10,  8,  4, 11,  8,  6,  7, 16,  0,  0,  0,  0,  0],
        [ 3,  2,  9,  3, 17,  9,  2, 16,  8, 12, 11,  0,  0],
        [13,  2, 16,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [15,  7, 12, 14, 16,  1,  3,  8, 14,  3, 17,  8, 16]]) torch.Size([5, 13])
tensor([[ 7, 10,  3, 13, 15],
        [ 5,  8,  2,  2,  7],
        [ 9,  4,  9, 16, 12],
        [ 9, 11,  3,  7, 14],
        [12,  8, 17,  0, 16],
        [ 1,  6,  9,  0,  1],
        [18,  7,  2,  0,  3],
        [12, 16, 16,  0,  8],
        [14,  0,  8,  0, 14],
        [ 9,  0, 12,  0,  3],
        [ 4,  0, 11,  0, 17],
        [ 0,  0,  0,  0,  8],
        [ 0,  0,  0,  0, 16]]) torch.Size([13, 5])


## pack_sequence
* link : https://pytorch.org/docs/stable/nn.html#pack-sequence

In [4]:
sorted_indices = sorted(indices, key=lambda tensor: tensor.size()[0], reverse=True)
pprint(sorted_indices)

[tensor([15,  7, 12, 14, 16,  1,  3,  8, 14,  3, 17,  8, 16]),
 tensor([ 7,  5,  9,  9, 12,  1, 18, 12, 14,  9,  4]),
 tensor([ 3,  2,  9,  3, 17,  9,  2, 16,  8, 12, 11]),
 tensor([10,  8,  4, 11,  8,  6,  7, 16]),
 tensor([13,  2, 16,  7])]


In [5]:
packed_indices = pack_sequence(sorted_indices)

print(packed_indices)
print(type(packed_indices))

PackedSequence(data=tensor([15,  7,  3, 10, 13,  7,  5,  2,  8,  2, 12,  9,  9,  4, 16, 14,  9,  3,
        11,  7, 16, 12, 17,  8,  1,  1,  9,  6,  3, 18,  2,  7,  8, 12, 16, 16,
        14, 14,  8,  3,  9, 12, 17,  4, 11,  8, 16]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]))
<class 'torch.nn.utils.rnn.PackedSequence'>


## pad_packed_sequence
* link : https://pytorch.org/docs/stable/nn.html#pad-packed-sequence

In [6]:
padded_indices_wbf_tuple = pad_packed_sequence(packed_indices, batch_first=True)
padded_indices_wobf_tuple = pad_packed_sequence(packed_indices, batch_first=False)

print(padded_indices_wbf_tuple)
print(padded_indices_wobf_tuple)

(tensor([[15,  7, 12, 14, 16,  1,  3,  8, 14,  3, 17,  8, 16],
        [ 7,  5,  9,  9, 12,  1, 18, 12, 14,  9,  4,  0,  0],
        [ 3,  2,  9,  3, 17,  9,  2, 16,  8, 12, 11,  0,  0],
        [10,  8,  4, 11,  8,  6,  7, 16,  0,  0,  0,  0,  0],
        [13,  2, 16,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0]]), tensor([13, 11, 11,  8,  4]))
(tensor([[15,  7,  3, 10, 13],
        [ 7,  5,  2,  8,  2],
        [12,  9,  9,  4, 16],
        [14,  9,  3, 11,  7],
        [16, 12, 17,  8,  0],
        [ 1,  1,  9,  6,  0],
        [ 3, 18,  2,  7,  0],
        [ 8, 12, 16, 16,  0],
        [14, 14,  8,  0,  0],
        [ 3,  9, 12,  0,  0],
        [17,  4, 11,  0,  0],
        [ 8,  0,  0,  0,  0],
        [16,  0,  0,  0,  0]]), tensor([13, 11, 11,  8,  4]))


## pack_padded_sequence
* link : https://pytorch.org/docs/stable/nn.html#pack-padded-sequence

In [7]:
print(pack_padded_sequence(*padded_indices_wbf_tuple, batch_first=True))
print(pack_padded_sequence(*padded_indices_wobf_tuple, batch_first=False))

PackedSequence(data=tensor([15,  7,  3, 10, 13,  7,  5,  2,  8,  2, 12,  9,  9,  4, 16, 14,  9,  3,
        11,  7, 16, 12, 17,  8,  1,  1,  9,  6,  3, 18,  2,  7,  8, 12, 16, 16,
        14, 14,  8,  3,  9, 12, 17,  4, 11,  8, 16]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]))
PackedSequence(data=tensor([15,  7,  3, 10, 13,  7,  5,  2,  8,  2, 12,  9,  9,  4, 16, 14,  9,  3,
        11,  7, 16, 12, 17,  8,  1,  1,  9,  6,  3, 18,  2,  7,  8, 12, 16, 16,
        14, 14,  8,  3,  9, 12, 17,  4, 11,  8, 16]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]))
